In [34]:
import pickle
import pandas as pd
import os

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [41]:
categorical = ['PULocationID', 'DOLocationID']
year = 2023
month = 3
taxi_type = 'yellow'
output_file = f'./output/{taxi_type}/{year:04d}_{month:02d}.parquet'

In [22]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [23]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [21]:
#Q1 Notebook
# What's the standard deviation of the predicted duration for this dataset?

predict_std = y_pred.std()
print(f'std of predicted duration is {predict_std:.3f}')

std of predicted duration is 6.247


In [50]:
#Q2 Preparing the output
# What's the size of the output file? 

# Creating a new column for ride_id
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

# Creating a new df+cols
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

# Saving result_df to a pq file
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

size_output_file = !du -sh {output_file} 
print(size_output_file[0])

66M	./output/yellow/2023_03.parquet
